In [ ]:
%run ./utility_functions.ipynb

In [ ]:
strategy = tf.distribute.MirroredStrategy(devices = ["/gpu:0", "/gpu:1", "/gpu:2", "/gpu:3"])

# Load Dataset

In [ ]:
image_shape = (96,96,3)

In [ ]:
train_dataset_path = "../Dataset/clean/CelebrityFaces/"
valid_dataset_path = "../Dataset/clean/FootballFaces/"

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

trainLoader = Loader(path=train_dataset_path,
                     image_shape=image_shape,
                     batch_size=BATCH_SIZE,
                     max_pos_pairs=3000,
                     apply_augmentation=True,
                     preprocess_pipeline = None,
                     normalize = True)

training_dataset = trainLoader.dataset

validLoader = Loader(path=valid_dataset_path,
                     image_shape=image_shape,
                     batch_size=BATCH_SIZE,
                     max_pos_pairs=500,
                     preprocess_pipeline = None,
                     normalize=True)

validation_dataset = validLoader.dataset

In [ ]:
trainLoader.visualize(value_range=(0,1),
                      color_mode_switch=False)

In [ ]:
embedding_dataset, class_names = validLoader.create_embedding_dataset()

# Training

## Simple Feature Extractor

In [ ]:
log_dir = f"../runs/{datetime.now().strftime('%Y%m%d=%H%M%S')}"
callb = callBacks(log_dir, embedding_dataset, class_names)

with strategy.scope(): 
    
    input_emb = tf.keras.Input(shape=image_shape)

    x = tf.keras.layers.Conv2D(32, (3,3), strides=2)(input_emb)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)

    x = tf.keras.layers.Conv2D(64, (3,3), strides=2)(x)  # Strided conv replaces pooling
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)

    x = tf.keras.layers.Conv2D(128, (3,3), strides=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)

    # x = tf.keras.layers.Conv2D(256, (3,3), strides=2)(x)
    # x = tf.keras.layers.BatchNormalization()(x)
    # x = tf.keras.layers.Activation("relu")(x)


    feature_extractor = Model(inputs=input_emb, outputs=x, name="feature_extractor")
    
    optimizer = Adam(learning_rate=0.0001)
    loss_fc = contrastiveLoss(margin=0.5, reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE )
    
    s_network = siameseNetwork(feature_extractor, image_shape, distance_metric="euclidean_distance", embedding_size=128)

    s_network.compile(optimizer=optimizer,
                      loss = loss_fc)
    
    s_network.fit(training_dataset, epochs=50, validation_data=validation_dataset, callbacks=[callb])

## MobileNet

In [ ]:
base_model = MobileNetV3Large(include_top = False,
                              weights = None,
                              input_shape=image_shape,
                              alpha = 1.0,
                              pooling = None,
                              dropout_rate = 0.1)

# num_layers = len(base_model.layers)
# freeze_count = int(0.30 * num_layers)

# for i, layer in enumerate(base_model.layers):
#     if i < freeze_count:
#         layer.trainable = False  # freeze
#     else:
#         layer.trainable = True   # unfreeze

In [ ]:
# embedding_model = s_network.embedding_model
log_dir = f"../runs/{datetime.now().strftime('%Y%m%d=%H%M%S')}"
callb = callBacks(log_dir, embedding_dataset, class_names)

with strategy.scope(): 
    optimizer = Adam(learning_rate=0.00001)
    loss_fc = contrastiveLoss(margin=0.5, reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE )
    
    feature_extractor = MobileNetV3Large(include_top = False,
                                        weights = None,
                                        input_shape=image_shape,
                                        alpha = 1.0,
                                        pooling = None,
                                        dropout_rate = 0.1)
    
    s_network = siameseNetwork(feature_extractor, image_shape, distance_metric="cosine_distance", embedding_size=512)

    s_network.compile(optimizer=optimizer,
                      loss = loss_fc)
    
    s_network.fit(training_dataset, epochs=15, validation_data=validation_dataset, callbacks=[callb])

## Nasnet

In [ ]:
base_model = NASNetMobile(input_shape=image_shape,
                         include_top=False,
                         weights=None,
                         input_tensor=None,
                         pooling=None)

In [ ]:
l2_reg = regularizers.l2(0.01)
base_embedding = createEmbedding(base_model, image_shape)
siamese_network = createSiameseNetwork(base_embedding, image_shape)

siamese_network.summary()

In [ ]:
optimizer = Adam(learning_rate=0.0001)
loss_fc = contrastive_loss
acc_metric = BinaryAccuracy(threshold=0.5)

fit(model = siamese_network,
    embedding_model = base_embedding,
    training_dataset=training_dataset,
    validation_dataset=validation_dataset,
    embedding_dataset = embedding_dataset,
    class_names = class_names,
    epochs = 30,
    loss_fc = loss_fc ,
    optimizer = optimizer,
    acc_metric = acc_metric)

## Resnet

In [ ]:
# embedding_model = s_network.embedding_model
log_dir = f"../runs/{datetime.now().strftime('%Y%m%d=%H%M%S')}"
callb = callBacks(log_dir, embedding_dataset, class_names)

with strategy.scope(): 
    optimizer = Adam(learning_rate=0.0001)
    loss_fc = contrastiveLoss(margin=0.5, reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE )
    # loss_fc = circleLoss(margin = 0.25, gamma = 64, reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
    
    feature_extractor = ResNet50V2(input_shape=image_shape,
                                   include_top=False,
                                   weights="imagenet",
                                   input_tensor=None,
                                   pooling=None)
    
    # for layer in feature_extractor.layers[:60]:
    #     layer.trainable = False
    
    s_network = siameseNetwork(feature_extractor, image_shape, distance_metric="cosine_distance", embedding_size=512)

    s_network.compile(optimizer=optimizer,
                      loss = loss_fc)
    
    s_network.fit(training_dataset, epochs=15, validation_data=validation_dataset, callbacks=[callb])

## VGG

In [ ]:
base_model = VGG16(input_shape=image_shape,
                         include_top=False,
                         weights=None,
                         input_tensor=None,
                         pooling=None)

In [ ]:
base_embedding = createEmbedding(base_model, image_shape)
base_embedding.summary()
siamese_network = createSiameseNetwork(base_embedding, image_shape)
siamese_network.summary()

In [ ]:
optimizer = Adam(learning_rate=0.0001)
loss_fc = contrastive_loss
acc_metric = BinaryAccuracy(threshold=0.5)

fit(model = siamese_network,
    embedding_model = base_embedding,
    training_dataset=training_dataset,
    validation_dataset=validation_dataset,
    embedding_dataset = embedding_dataset,
    class_names = class_names,
    epochs = 100,
    loss_fc = loss_fc ,
    optimizer = optimizer,
    acc_metric = acc_metric)

##  Evaluating

In [ ]:
old_weights = siamese_network.get_weights()
path = "../runs/20250204-102132/siamese_model_epoch_20.h5"
siamese_network.load_weights(path)
new_weights = siamese_network.get_weights()

print("Mean of old weights[0]:", old_weights[0].mean())
print("Mean of new weights[0]:", new_weights[0].mean())

In [ ]:
training_dataset
validation_dataset

In [ ]:
acc = evaluate(siamese_network, validation_dataset, 0.39) #0.24 on training set
acc